In [ ]:
# load libraries
import tensorflow as tf
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from scipy.stats import mode
import numpy as np

In [ ]:
# Load the data and train-teat split
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train, y_train), (X_test, y_test) = fashion_mnist

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
X_train.shape

(60000, 28, 28)

In [ ]:
# Reshaping the Data set for clusteing
X_train = X_train.reshape(60000,784)
X_train.shape

(60000, 784)

In [ ]:
def new_labels(k,data,labels,p):
  Kmeans = KMeans(k,)
  Kmeans.fit(data)
  cluster_labels = Kmeans.labels_
  centroids = Kmeans.cluster_centers_

  # Find 10 nearest points for each centroid
  num_neighbors = 10
  nbrs = NearestNeighbors(n_neighbors=num_neighbors).fit(data)
  distances, indices = nbrs.kneighbors(centroids)

  # Assign labels to clusters based on mode of 10 nearest points
  cluster_labels_mode = []
  for i in range(k):
      nearest_labels = [ labels[idx] for idx in indices[i] ]
      cluster_label_mode = mode(nearest_labels)[0]
      cluster_labels_mode.append(cluster_label_mode)

  # Finding 40% nearest neighbour
  num_neighbors2 = np.ceil(p * len(data) / k).astype(int)
  nbrs2 = NearestNeighbors(n_neighbors=num_neighbors2).fit(data)
  distances, indices2 = nbrs2.kneighbors(centroids)
  indices2 = indices2.reshape(k*num_neighbors2)

  # Assign labels to each point in data based on cluster labels mode
  new_labels = np.array([cluster_labels_mode[label] for label in cluster_labels])

  # cuting to 40% nearest
  new_data = data[indices2]
  new_labels = new_labels[indices2]
  old_labels = labels[indices2]
  return new_data, new_labels, old_labels

def wrong_labels(y,y_propagated):
     num_wrong_labels = np.sum(y_propagated != y)
     print('Number of wrong labels after label propagation:', num_wrong_labels)


In [ ]:
# Checking
for i in [50,100,200,300]:
  new_X_train, new_y_train, old_y_train = new_labels(i,X_train,y_train,0.4)
  new_X_train = new_X_train/255
  new_X_train = new_X_train.reshape((24000,28,28))
  X_mod = new_X_train[:-5000]
  y_mod = new_y_train[:-5000]
  X_val, y_val = new_X_train[-5000:], new_y_train[-5000:]

  # Train model on new labels
  model = tf.keras.Sequential([
      tf.keras.layers.Flatten(input_shape=[28, 28]),
      tf.keras.layers.Dense(100, activation="relu"),
      tf.keras.layers.Dense(50, activation="relu"),
      tf.keras.layers.Dense(10, activation="softmax")
  ])
  model.compile(loss="sparse_categorical_crossentropy",optimizer="sgd",metrics=["accuracy"])

  history = model.fit(new_X_train, new_y_train, epochs=10 ,validation_data=(X_val, y_val))

  # Evaluate the results
  accuracy = model.evaluate(X_test/255, y_test)[1]
  print( "For k = ",i," model is giving ", accuracy, "accuracy")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Epoch 1/10
750/750 [==============================] - 3s 4ms/step - loss: 0.6731 - accuracy: 0.7985 - val_loss: 0.3891 - val_accuracy: 0.8524
Epoch 2/10
750/750 [==============================] - 3s 4ms/step - loss: 0.2980 - accuracy: 0.8908 - val_loss: 0.2803 - val_accuracy: 0.8950
Epoch 3/10
750/750 [==============================] - 4s 6ms/step - loss: 0.2376 - accuracy: 0.9103 - val_loss: 0.1989 - val_accuracy: 0.9254
Epoch 4/10
750/750 [==============================] - 2s 3ms/step - loss: 0.2108 - accuracy: 0.9183 - val_loss: 0.2129 - val_accuracy: 0.9150
Epoch 5/10
750/750 [==============================] - 2s 3ms/step - loss: 0.1946 - accuracy: 0.9220 - val_loss: 0.2422 - val_accuracy: 0.8956
Epoch 6/10
750/750 [==============================] - 2s 3ms/step - loss: 0.1830 - accuracy: 0.9270 - val_loss: 0.2683 - val_accuracy: 0.8766
Epoch 7/10
750/750 [==============================] - 2s 3ms/step - loss: 0.1744 - accuracy: 0.9300 - val_loss: 0.2148 - val_accuracy: 0.9080
Epoch 

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Epoch 1/10
750/750 [==============================] - 3s 4ms/step - loss: 0.7578 - accuracy: 0.7788 - val_loss: 0.3851 - val_accuracy: 0.8494
Epoch 2/10
750/750 [==============================] - 2s 3ms/step - loss: 0.3225 - accuracy: 0.8877 - val_loss: 0.2757 - val_accuracy: 0.9108
Epoch 3/10
750/750 [==============================] - 4s 5ms/step - loss: 0.2513 - accuracy: 0.9119 - val_loss: 0.2107 - val_accuracy: 0.9262
Epoch 4/10
750/750 [==============================] - 2s 3ms/step - loss: 0.2192 - accuracy: 0.9202 - val_loss: 0.1995 - val_accuracy: 0.9274
Epoch 5/10
750/750 [==============================] - 2s 3ms/step - loss: 0.2001 - accuracy: 0.9256 - val_loss: 0.1671 - val_accuracy: 0.9392
Epoch 6/10
750/750 [==============================] - 3s 3ms/step - loss: 0.1872 - accuracy: 0.9301 - val_loss: 0.1669 - val_accuracy: 0.9386
Epoch 7/10
750/750 [==============================] - 2s 3ms/step - loss: 0.1782 - accuracy: 0.9320 - val_loss: 0.1627 - val_accuracy: 0.9416
Epoch 

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Epoch 1/10
750/750 [==============================] - 3s 4ms/step - loss: 0.8472 - accuracy: 0.7325 - val_loss: 0.5620 - val_accuracy: 0.7820
Epoch 2/10
750/750 [==============================] - 3s 4ms/step - loss: 0.4297 - accuracy: 0.8488 - val_loss: 0.3653 - val_accuracy: 0.8596
Epoch 3/10
750/750 [==============================] - 4s 5ms/step - loss: 0.3486 - accuracy: 0.8740 - val_loss: 0.3096 - val_accuracy: 0.8824
Epoch 4/10
750/750 [==============================] - 3s 3ms/step - loss: 0.3105 - accuracy: 0.8864 - val_loss: 0.2673 - val_accuracy: 0.9014
Epoch 5/10
750/750 [==============================] - 2s 3ms/step - loss: 0.2869 - accuracy: 0.8926 - val_loss: 0.2503 - val_accuracy: 0.9006
Epoch 6/10
750/750 [==============================] - 2s 3ms/step - loss: 0.2661 - accuracy: 0.8980 - val_loss: 0.2453 - val_accuracy: 0.9020
Epoch 7/10
750/750 [==============================] - 3s 4ms/step - loss: 0.2541 - accuracy: 0.9033 - val_loss: 0.2308 - val_accuracy: 0.9096
Epoch 

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Epoch 1/10
750/750 [==============================] - 4s 5ms/step - loss: 0.8406 - accuracy: 0.7523 - val_loss: 0.4881 - val_accuracy: 0.8358
Epoch 2/10
750/750 [==============================] - 3s 4ms/step - loss: 0.3815 - accuracy: 0.8763 - val_loss: 0.3729 - val_accuracy: 0.8894
Epoch 3/10
750/750 [==============================] - 2s 3ms/step - loss: 0.3083 - accuracy: 0.8940 - val_loss: 0.3202 - val_accuracy: 0.8908
Epoch 4/10
750/750 [==============================] - 3s 3ms/step - loss: 0.2754 - accuracy: 0.9018 - val_loss: 0.2833 - val_accuracy: 0.9024
Epoch 5/10
750/750 [==============================] - 3s 4ms/step - loss: 0.2557 - accuracy: 0.9067 - val_loss: 0.2664 - val_accuracy: 0.9062
Epoch 6/10
750/750 [==============================] - 3s 4ms/step - loss: 0.2393 - accuracy: 0.9128 - val_loss: 0.2485 - val_accuracy: 0.9100
Epoch 7/10
750/750 [==============================] - 3s 3ms/step - loss: 0.2284 - accuracy: 0.9151 - val_loss: 0.2195 - val_accuracy: 0.9176
Epoch 